# Miniproject 1: Image Classification

## Introduction

### Important dates:

- Project release: Friday, 15th March 2019
- **Submission deadline**: Monday, 29th April 2019, 11:59 pm

### Description

One of the deepest traditions in learning about deep learning is to first [tackle the exciting problem of MNIST classification](http://yann.lecun.com/exdb/mnist/). [The MNIST database](https://en.wikipedia.org/wiki/MNIST_database) (Modified National Institute of Standards and Technology database) is a large database of handwritten digits that is commonly used as a first test for new classification algorithms. 
We follow this tradition to investigate the performance of artificial neural networks of different complexity on MNIST. However, since MNIST is too easy for accessing the full power of modern machine learning algorithms (see e.g. [this post](https://twitter.com/goodfellow_ian/status/852591106655043584)) we will extend our analysis to the recently introduced, harder [Fashion-MNIST dataset](https://github.com/zalandoresearch/fashion-mnist).


### Prerequisites

- You should have a running installation of [tensorflow](https://www.tensorflow.org/install/) and [keras](https://keras.io/). Feel free to gain inspiration from the [Keras example directory](https://github.com/keras-team/keras/tree/master/examples) for your implementations.
- You should know the concepts "multilayer perceptron", "stochastic gradient descent with minibatches", "convolutional neural network", "training and validation data", "overfitting" and "early stopping".

### What you will learn

- You will learn how to define feedforward neural networks in keras and fit them to data.
- You will be guided through a prototyping procedure for the application of deep learning to a specific domain.
- You will get in contact with concepts discussed later in the lecture, like "regularization", "batch normalization" and "convolutional networks".
- You will gain some experience on the influence of network architecture, optimizer and regularization choices on the goodness of fit.
- You will learn to be more patient :) Some fits may take your computer quite a bit of time; run them over night (or on an external server).

### Evaluation criteria

The evaluation is (mostly) based on the figures you submit and your answer sentences. Provide clear and concise answers respecting the indicated maximum length (answers to the questions should be below the line that says "Answer to question ...").

**The submitted notebook must be run by you!** We will only do random tests of your code and not re-run the full notebook. There will be fraud detection sessions at the end of the semester.

### Your names

**Before you start**: please enter your full name(s) in the field below.

In [ ]:
student1 = "Francesco Pase"
student2 = "Murat  Topak"

## Some helper functions

For your convenience we provide here some functions to preprocess the data and plot the results later. Simply run the following cells with `Shift-Enter`.

### Dependencies and constants

In [ ]:
%matplotlib inline

import numpy as np
import time
import matplotlib.pyplot as plt
import scipy.io

import keras
from keras import regularizers
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten, BatchNormalization
from keras.optimizers import SGD, Adam

### Plotting

In [ ]:
def plot_some_samples(x, y = [], yhat = [], select_from = [], 
                      ncols = 6, nrows = 4, xdim = 28, ydim = 28,
                      label_mapping = range(10)):
    """plot some input vectors as grayscale images (optionally together with their assigned or predicted labels).
    
    x is an NxD - dimensional array, where D is the length of an input vector and N is the number of samples.
    Out of the N samples, ncols x nrows indices are randomly selected from the list select_from (if it is empty, select_from becomes range(N)).
    
    Keyword arguments:
    y             -- corresponding labels to plot in green below each image.
    yhat          -- corresponding predicted labels to plot in red below each image.
    select_from   -- list of indices from which to select the images.
    ncols, nrows  -- number of columns and rows to plot.
    xdim, ydim    -- number of pixels of the images in x- and y-direction.
    label_mapping -- map labels to digits.
    
    """
    fig, ax = plt.subplots(nrows, ncols)
    if len(select_from) == 0:
        select_from = range(x.shape[0])
    indices = np.random.choice(select_from, size = min(ncols * nrows, len(select_from)), replace = False)
    for i, ind in enumerate(indices):
        thisax = ax[i//ncols,i%ncols]
        thisax.matshow(x[ind].reshape(xdim, ydim), cmap='gray')
        thisax.set_axis_off()
        if len(y) != 0:
            j = y[ind] if type(y[ind]) != np.ndarray else y[ind].argmax()
            thisax.text(0, 0, (label_mapping[j]+1)%10, color='green', 
                                                       verticalalignment='top',
                                                       transform=thisax.transAxes)
        if len(yhat) != 0:
            k = yhat[ind] if type(yhat[ind]) != np.ndarray else yhat[ind].argmax()
            thisax.text(1, 0, (label_mapping[k]+1)%10, color='red',
                                             verticalalignment='top',
                                             horizontalalignment='right',
                                             transform=thisax.transAxes)
    return fig

def prepare_standardplot(title, xlabel):
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12,6))
    fig.suptitle(title)
    ax1.set_ylabel('categorical cross entropy')
    ax1.set_xlabel(xlabel)
    ax1.set_yscale('log')
    ax2.set_ylabel('accuracy [% correct]')
    ax2.set_xlabel(xlabel)
    return fig, ax1, ax2

def finalize_standardplot(fig, ax1, ax2):
    ax1handles, ax1labels = ax1.get_legend_handles_labels()
    if len(ax1labels) > 0:
        ax1.legend(ax1handles, ax1labels)
    ax2handles, ax2labels = ax2.get_legend_handles_labels()
    if len(ax2labels) > 0:
        ax2.legend(ax2handles, ax2labels)
    fig.tight_layout()
    plt.subplots_adjust(top=0.9)

def plot_history(history, title):
    fig, ax1, ax2 = prepare_standardplot(title, 'epoch')
    ax1.plot(history.history['loss'], label = "training")
    ax1.plot(history.history['val_loss'], label = "validation")
    ax2.plot(history.history['acc'], label = "training")
    ax2.plot(history.history['val_acc'], label = "validation")
    finalize_standardplot(fig, ax1, ax2)
    
    return fig


## Exercise 1: Data import and visualization (4 points)

### Description

### Loading the data

The datasets we use in this project (MNIST, Fashion-MNIST) consists of grayscale images with 28x28 pixels. Keras comes with a convenient in-built [data importer](https://keras.io/datasets/) for common datasets.

1. As a warm-up exercise, use this importer to (down-)load the MNIST and Fashion-MNIST dataset. Assign useful variables to test & train images and labels for both datasets respectively. (2 pts)
2. Use the corresponding plotting function defined above to plot some samples of the two datasets. What do the green digits at the bottom left of each image indicate? (1 sentence max.) (2 pts)

The low resolution (and grayscale) of the images certainly misses some information that could be helpful for classifying the images. However, since the data has lower dimensionality due to the low resolution, the fitting procedures converge faster. This is an advantage in situations like here (or generally when prototyping), were we want to try many different things without having to wait too long for computations to finish.



### Solution

In [ ]:
from keras.datasets import mnist
from keras.datasets import fashion_mnist

(x_train, y_train), (x_test, y_test) =  mnist.load_data()
(x_fashion_train, y_fashion_train), (x_fashion_test, y_fashion_test) = fashion_mnist.load_data()

plot_some_samples(x_train, y_train)
plot_some_samples(x_fashion_train, y_fashion_train);

Answer to question 2:

The green digits are the labels of the plotted input vectors.

**Data pre-processing**: To prepare for fitting we transform the labels to one hot coding, i.e. for 5 classes, label 2 becomes the vector [0, 0, 1, 0, 0] (python uses 0-indexing). Furthermore we reshape (flatten) the input images to input vectors and rescale the data into the range [0,1].

In [ ]:
y_train = keras.utils.to_categorical(y_train)
y_test = keras.utils.to_categorical(y_test)

y_fashion_train = keras.utils.to_categorical(y_fashion_train)
y_fashion_test = keras.utils.to_categorical(y_fashion_test)

In [ ]:
x_train = x_train.reshape(x_train.shape[0], x_train.shape[1]*x_train.shape[2])/np.max(x_train)
x_test = x_test.reshape(x_test.shape[0], x_test.shape[1]*x_test.shape[2])/np.max(x_test)

x_fashion_train = x_fashion_train.reshape(x_fashion_train.shape[0], x_fashion_train.shape[1]*x_fashion_train.shape[2])/np.max(x_fashion_train)
x_fashion_test = x_fashion_test.reshape(x_fashion_test.shape[0], x_fashion_test.shape[1]*x_fashion_test.shape[2])/np.max(x_fashion_test)

## Exercise 2: No hidden layer (10 points)

### Description

Define and fit a model without a hidden layer (since we will use multi-layer models later in this project, you can define a general constructor function for models with an arbitrary number of hidden layers already at this point). (1 pt for each step)

1. Use the softmax activation for the output layer.
2. Use the categorical_crossentropy loss.
3. Add the accuracy metric to the metrics.
4. Choose stochastic gradient descent for the optimizer.
5. Choose a minibatch size of 128.
6. Fit for as many epochs as needed to see no further decrease in the validation loss.
7. Plot the output of the fitting procedure (a history object) using the function plot_history defined above.
8. Determine the indices of all test images that are misclassified by the fitted model and plot some of them using the function 
   `plot_some_samples(x_test, y_test, yhat_test, error_indices)`. Explain the green and red digits at the bottom of each image.
9. Repeat the above steps for fitting the network to the Fashion-MNIST dataset.


Hints:
* Read the keras docs, in particular [Getting started with the Keras Sequential model](https://keras.io/getting-started/sequential-model-guide/).
* Have a look at the keras [examples](https://github.com/keras-team/keras/tree/master/examples), e.g. [mnist_mlp](https://github.com/keras-team/keras/blob/master/examples/mnist_mlp.py).

### Solution

In [ ]:
#MNIST

from keras.models import Sequential
from keras.layers import Dense, Activation

model = Sequential()
model.add(Dense(10, input_dim = 784, activation='softmax'))


model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, batch_size=128, epochs=50, validation_split=0.2, verbose=1)


pred = model.predict(x_test)
y_hat_test = np.argmax(pred, axis=1)
y_truth = np.argmax(y_test, axis=1)
err_indices = (y_hat_test != y_truth).nonzero()

plot_history(history, 'MNIST');
plot_some_samples(x_test, y_test, y_hat_test, err_indices[0]);



In [ ]:
#MNIST FASHION
model = Sequential()
model.add(Dense(10, input_dim = 784, activation='softmax'))


model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_fashion_train, y_fashion_train, batch_size=128, epochs=50, validation_split=0.2, verbose=1)


pred = model.predict(x_fashion_test)
y_hat_fashion_test = np.argmax(pred, axis=1)
y_fashion_truth = np.argmax(y_fashion_test, axis=1)
err_indices = (y_hat_fashion_test != y_fashion_truth).nonzero()

plot_history(history, 'Fashion-MNIST');
plot_some_samples(x_fashion_test, y_fashion_test, y_hat_fashion_test, err_indices[0]);

Answer to question 8:

They are both labels, the green being the true label, and the red being the predicted label. For example, in the MNIST dataset, we see a lot of 8,9,0 are misclassified. This makes sense since those 3 numbers highly resemble each other in terms of their shape and drawing. Of course, we can try to fine-tune the model or simply retrieve more data to reduce these mispredictions.

## Exercise 3: One hidden layer, different optizimizers & overfitting (10 points)

### Description

Train a network with one hidden layer and compare different optimizers.

1. Use one hidden layer with 128 units and the 'relu' activation. Use the [summary method](https://keras.io/models/about-keras-models/) to display your model in a compact way. (1 pt)
2. Fit the model for 50 epochs with different learning rates of stochastic gradient descent (SGD). (1pt)
3. Replace the stochastic gradient descent optimizer with the [Adam optimizer](https://keras.io/optimizers/#adam). (1pt)
4. Plot the learning curves of SGD with a reasonable learning rate (i.e. in the range [0.01,0.1]) together with the learning curves of Adam in the same figure. Take care of a reasonable labeling of the curves in the plot. (2pts)
5. Answer the questions below. (4pts)
6. Run the network (using the Adam optimizer) on the Fashion-MNIST dataset and plot the learning curves using the plot_history function defined above. (1pt)

### Solution

In [ ]:
from keras import optimizers


In [ ]:
from keras import optimizers

model = Sequential()
model.add(Dense(128, input_dim = 784, activation='relu'))
model.add(Dense(10, activation='softmax'))

model.summary()


available_lr = [0.05, 0.9,  0.0001]
histories = []
for alr in available_lr:
    sgd = optimizers.SGD(lr = alr)
    model.compile(optimizer = sgd, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    history = model.fit(x_fashion_train, y_fashion_train, batch_size = 128, epochs = 50, validation_split = 0.2, verbose=1)
    histories.append(history)


In [ ]:
def comparisonSGD_plot(histories, labels, title):
    fig, ax1, ax2 = prepare_standardplot(title, "epochs")
    linestyles = [':', '--', '-', ':']
    for (history,lbl, lstyle) in zip(histories, labels, linestyles):
        ax1.plot(history.history['loss'], label= str(lbl) + ' training', linestyle=lstyle)
        ax1.plot(history.history['val_loss'], label= str(lbl) + ' validation', linestyle=lstyle)
        ax2.plot(history.history['acc'], label= str(lbl) + ' training', linestyle=lstyle)
        ax2.plot(history.history['val_acc'], label= str(lbl) + ' validation', linestyle=lstyle)

    fig.suptitle(title, x=0.8)
    ax2handles, ax2labels = ax2.get_legend_handles_labels()
    if len(ax2labels) > 0:
        ax2.legend(ax2handles, ax2labels, loc='upper right', bbox_to_anchor=(0.5, -0.2), ncol=3)
    plt.subplots_adjust(right=1.5)
    #fig.tight_layout()
    return fig
comparisonSGD_plot(histories, available_lr, 'VARIOUS LRs');

In [ ]:
# This plotting routine might help you ...
from keras import optimizers
def comparison_plot(history_sgd, history_adam, label1, label2, title):
    fig, ax1, ax2 = prepare_standardplot(title, "epochs")
    ax1.plot(history_sgd.history['loss'], label=label1 + ' training')
    ax1.plot(history_sgd.history['val_loss'], label=label1 + ' validation')
    ax1.plot(history_adam.history['loss'], label=label2 + ' training')
    ax1.plot(history_adam.history['val_loss'], label=label2 + ' validation')
    ax2.plot(history_sgd.history['acc'], label=label1 + ' training')
    ax2.plot(history_sgd.history['val_acc'], label=label1 + ' validation')
    ax2.plot(history_adam.history['acc'], label=label2 + ' training')
    ax2.plot(history_adam.history['val_acc'], label=label2 + ' validation')
    finalize_standardplot(fig, ax1, ax2)
    return fig




In [ ]:
model = Sequential()
model.add(Dense(128, input_dim = 784, activation='relu'))
model.add(Dense(10, activation='softmax'))

#model.summary()

sgd = optimizers.SGD(lr = 0.01)
model.compile(optimizer = sgd, loss = 'categorical_crossentropy', metrics = ['accuracy'])
historySGD = model.fit(x_train, y_train, batch_size = 128, epochs = 50, validation_split = 0.2, verbose=1)

adam = optimizers.Adam(lr = 0.01)
model.compile(optimizer=adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
historyAdam = model.fit(x_train, y_train, batch_size = 128, epochs = 50, validation_split = 0.2, verbose=1)


_ = comparison_plot(historySGD, historyAdam, 'SGD', 'Adam', 'SGD vs Adam with lr=0.01');

In [ ]:
model = Sequential()
model.add(Dense(128, input_dim = 784, activation='relu'))
model.add(Dense(10, activation='softmax'))


adam = optimizers.Adam(lr = 0.01)
model.compile(optimizer=adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
historyAdam = model.fit(x_fashion_train, y_fashion_train, batch_size = 128, epochs = 50, validation_split = 0.2, verbose=1)
plot_history(historyAdam, 'Adam on Fashion-MNIST');

**Question**: What happens if the learning rate of SGD is A) very large B) very small? Please answer A) and B) with one full sentence each (double click this markdown cell to edit).

**Answer**:

A) In the first figure, we see that the model follows a fluctuated learning curve as opposed to the optimal learning rate as at each step it is likely to end up missing the local optimum due to high learning rate.

B) Again from the first figure we conclude that the model tends to underperform, meaning that it does not achieve the best it can due to low learning rate as at each step the weights tend to stay the same because of the low learning rate

**Question**: At which epoch (approximately) does the Adam optimizer start to overfit (on MNIST)? Please answer with one full sentence.

**Answer**: From the red curve in the last figure, we see the loss on validation set starts to increase at around epoch 5-10, indicating a possible overfit.

**Question**: Explain the qualitative difference between the loss curves and the accuracy curves with respect to signs of overfitting. Please answer with at most 3 full sentences.

TOOOOOOOOOOOOOOOOOOOOOOOOOOOO DOOOOOOOOOOOOOOOOOOO

**Answer**: We decided to analyze Adam experiment. The first difference is that validation Loss and Accuracy curves present different behaviours , so while the Loss one goes up above the training Loss, the Accuracy curve doesn't change much over time. This is most likely the case because while overfitting, the errors in the misclassified validation samples icrease but the prectentage of right and wrong predictions remained the same.

## Exercise 4: Model performance as a function of number of hidden neurons (8 points)

### Description

Investigate how the best validation loss and accuracy depends on the number of hidden neurons in a single layer.

1. Fit a reasonable number of models (e.g. 5) with different hidden layer sizes (between 10 and 1000 hidden neurons) to the MNIST dataset. You may use the Adam optimizer and a meaningful number of epochs (overfitting!). (3 pts)
2. Plot the best validation loss and accuracy versus the number of hidden neurons. Is the observed trend in accordance with the [general approximation theorem](https://en.wikipedia.org/wiki/Universal_approximation_theorem)? If not, what might be practical reasons for the deviation? (2 sentences max.) (3 pts)
3. Repeat steps 1. & 2. for the Fashion-MNIST dataset. (2 pts)

In this exercise we fit each model only for one initialization and random seed. In practice one would collect some statistics (e.g. 25-, 50-, 75-percentiles) for each layer size by fitting each model several times with different initializations and the random seeds. You may also want to do this here. It is a good exercise, but not mandatory as it takes quite a bit of computation time.

### Solution

In [ ]:
from keras import optimizers
def comparison_layer(histories,num_units, title):
    fig, ax1, ax2 = prepare_standardplot(title, "# of units")
    best_val_losses = [min(history.history['val_loss']) for history in histories]
    best_val_accs = [max(history.history['val_acc']) for history in histories]

    ax1.plot(num_units, best_val_losses, marker='o', linestyle='--')

    ax2.plot(num_units, best_val_accs, marker='o', linestyle='--')

    finalize_standardplot(fig, ax1, ax2)
    ax2.set_ylim([0.9,1])
    ax1.set_ylim([0.1,0.3])
    return fig


In [ ]:




num_units = [10, 20, 40, 80, 160, 320, 640, 1000]
histories = []
for u in num_units:
    model = Sequential()
    model.add(Dense(u, input_dim = 784, activation='relu'))
    model.add(Dense(10, activation='softmax'))
    adam = optimizers.Adam(lr = 0.01)
    model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    history = model.fit(x_train, y_train, batch_size = 128, epochs = 10, validation_split = 0.2, verbose=1)
    histories.append(history)




In [ ]:
comparison_layer(histories, num_units, 'Loss and Accuracy vs. # of Units');


Answer to question 2:

Indeed, the figures are in accordance with the general approximation theorem although through the end we see an upward trend in the loss plot. This is also expected since the data is 784-d but we even tried 1000 units, so it is most likely we overfitted in that area.

## Exercise 5: Going deeper: tricks and regularization (8 points)

### Description

Adding hidden layers to a deep network does not necessarily lead to a straight-forward improvement of performance. Overfitting can be counteracted with regularization and dropout. Batch normalization is supposed to mainly speed up convergence. Since the MNIST dataset is almost perfectly solved already by a one-hidden-layer network we use the Fashion-MNIST dataset in this exercise.

1. Add one or two hidden layers with 50 hidden neurons (each) and train the network for a sufficiently long time (at least 100 epochs). Since deep models are very expressive you will most probably encounter overfitting. Try to improve the best validation scores of the model (even if it is only a minor improvement) by experimenting with batch_normalization layers, dropout layers and l1- and l2-regularization on weights (kernels) and biases. (4 pts)
2. After you have found good settings, plot the learning curves for both models, naive (=no tricks/regularization) and tuned (=tricks + regularized), preferably together in a comparison plot. Discuss your results; refer to the model performance with only 1 hidden layer. (2 sentences max.) (2pts)
3. Fit your best performing (probably regularized deep) model also to MNIST for having a reference for the next exercise. Plot the resulting learning curves. (2 pts)

### Solution

In [ ]:
#Standard Model - No Tricks
histories = dict()
model = Sequential()
model.add(Dense(50, input_dim = 784, activation='relu'))

#model.add(Dense(50, input_dim = 50, activation='relu'))

model.add(Dense(10, activation='softmax'))
adam = optimizers.Adam(lr = 0.001)
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
history = model.fit(x_fashion_train, y_fashion_train, batch_size = 128, epochs = 100, validation_split = 0.2, verbose=1)
histories["NoTrick"] = history


In [ ]:
plot_history(histories["NoTrick"], "Simple Deep Model");

In [ ]:
#Batch Normalization

model = Sequential()
model.add(Dense(50, input_dim = 784, activation='linear'))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001))
model.add(Activation('relu'))

model.add(Dense(10, activation='linear'))
model.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.00001))
model.add(Activation('softmax'))
adam = optimizers.Adam(lr = 0.001)
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
history = model.fit(x_fashion_train, y_fashion_train, batch_size = 128, epochs = 100, validation_split = 0.2, verbose=1)
histories["BatchNorm"] = history




In [ ]:
plot_history(histories["BatchNorm"], "Batch Normalization");

In [ ]:
#Dropout

model = Sequential()
model.add(Dense(50, input_dim = 784, activation='relu'))
#model.add(BatchNormalization())
#model.add(Activation('relu'))
model.add(Dropout(rate=0.45))



model.add(Dense(10, activation='softmax'))
adam = optimizers.Adam(lr = 0.001)
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
history = model.fit(x_fashion_train, y_fashion_train, batch_size = 128, epochs = 100, validation_split = 0.2, verbose=1)



histories["Dropout"] = history




In [ ]:
plot_history(histories["Dropout"], "Dropout");

In [ ]:
#L1 Normalization
model = Sequential()

model.add(Dense(50, input_dim = 784, activation='relu', kernel_regularizer= regularizers.l1(0.0002), bias_regularizer = regularizers.l1(0.0002)))

model.add(Dense(10, activation='softmax'))
adam = optimizers.Adam(lr = 0.001)
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
history = model.fit(x_fashion_train, y_fashion_train, batch_size = 128, epochs = 100, validation_split = 0.2, verbose=1)
histories["L1-Norm"] = history




In [ ]:
plot_history(histories["L1-Norm"], "L1 Normalization lambda = 0.0002");

Answer to question 2 (comments):

In [ ]:
#L2 Normalization

model = Sequential()

model.add(Dense(50, input_dim = 784, activation='relu', kernel_regularizer= regularizers.l2(0.0008), bias_regularizer = regularizers.l2(0.0008)))

#model.add(Dense(50, input_dim = 50, activation='relu',  kernel_regularizer= regularizers.l2(0.001), bias_regularizer = regularizers.l2(0.001)))

model.add(Dense(10, activation='softmax'))
adam = optimizers.Adam(lr = 0.001)
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])
history = model.fit(x_fashion_train, y_fashion_train, batch_size = 128, epochs = 100, validation_split = 0.2, verbose=1)
histories["L2-Norm"] = history




In [ ]:
plot_history(histories["L2-Norm"], "L2 Normalization lambda = 0.0008");

In [ ]:
#Comparison

comparison_plot(histories["Dropout"], histories["NoTrick"], 'Dropout', 'No Tricks', 'Comparison between simple and dropout models');

## Exercise 6: Convolutional neural networks (CNNs) (10 points)

### Description

Convolutional neural networks have an inductive bias that is well adapted to image classification.

1. Design a convolutional neural network, play with different architectures and parameters. Hint: You may get valuable inspiration from the keras [examples](https://github.com/keras-team/keras/tree/master/examples). (4 pts)
2. Plot the learning curves of the convolutional neural network for MNIST and Fashion-MNIST. (4 pts)
3. How does the CNN performance compare to the so far best performing (deep) neural network model for the two data sets? (2 sentences max.) (2 pts)

### Solution

Answer to question 3:

## Exercise 7: Sigmoidal activation function and batch-normalization (6 points)

### Description:

In the original publication of batch normalization [Ioffe and Szegedy, 2014](https://arxiv.org/pdf/1502.03167.pdf), the authors mention a particularly beneficial effect of their method on networks with sigmoidal activation functions. This is because such networks usually suffer from saturating activations/vanishing gradients. Here we want to reproduce this behaviour (Chose either MNIST or Fashion-MNIST for this exercise).

1. Implement the same convolutional network as in the previous exercise, but using the sigmoid activation function instead of the standard choice ReLU. Train the network for a reasonable amount of time. What do you observe? (1 sentence max.) (3 pts)
2. Add batch-normalization layers to all convolutional and fully-connected layers (i.e. before each layer with learnable parameters). How does the performance change? Can the network reach the ReLU-CNN performance of the previous exercise? (1 sentence max.) (3 pts)
3. **BONUS (optional, not graded**): Investigate our initial guess that saturating activity/vanishing gradients might be the cause of this behaviour. For that, create histograms of the hidden activitions for different hidden layers for the sigmoid-CNN and the sigmoid-CNN with batch-normalization (counting over both, samples and neurons per layer). You may only chose layers with learnable parameters. What do you observe?
Hint: You can use the [keract](https://github.com/philipperemy/keract) package to access neural activation values for all layers of your network model.



### Solution:

Answer to question 1:

Answer to question 2: